# Tuning: Neuronales Netzwerk mit dem Tuner von Weights & Biases

In diesem Notebook wird ein neuronales Netzwerk mit dem [Tuner von Weights & Biases](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/tensorflow/Hyperparameter_Optimization_in_TensorFlow_using_W%26B_Sweeps.ipynb#scrollTo=crY5iadfKp_b) optimiert. Der Tuner von Weights & Biases ist ein Tool, das automatisiert die Hyperparameter eines Modells optimiert. In diesem Fall wird der Tuner verwendet, um die Batch Size als Hyperparameter zu optimieren. Dabei ist das Ziel den Validierungsfehler zu minimieren. Dieses Notebook dient als Grundlage für das Notebook `train_wandb_tuned.ipynb`.

Zunächst werden alle nötigen Bibliotheken importiert.

In [1]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import os
import wandb
import tqdm
import tensorflow as tf
import keras
import seaborn as sns

from tensorflow.keras import layers, Model, optimizers, losses, callbacks
from tensorflow import GradientTape
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
from wandb.integration.keras import WandbMetricsLogger
from sklearn.metrics import confusion_matrix

Anschließend können die verwendeten Datensätze geladen werden.

In [2]:
train = pd.read_csv('../data/preprocessed/train_data.csv')
test = pd.read_csv('../data/preprocessed/test_data.csv')
val = pd.read_csv('../data/preprocessed/val_data.csv')

In [3]:
train.shape, test.shape, val.shape

((50176, 48), (5469, 48), (10752, 48))

In [4]:
X_train = train.drop("Is high risk", axis=1)
Y_train = train["Is high risk"]

X_test = test.drop("Is high risk", axis=1)
Y_test = test["Is high risk"]

X_val = val.drop("Is high risk", axis=1)
Y_val = val["Is high risk"]

In [5]:
# wandb.login()

Nun werden die Funktionen für das neuronale Netzwerk definiert. Dabei wird ein einfaches Netzwerk mit vier versteckten Schichten mit jeweils 64 Neuronen und der ReLU-Aktivierungsfunktion verwendet. Weiterhin wird die Funktion für die Trainingsschritte und die Testschritte definiert.

In [6]:
def Model():
    x = layers.Input(shape=(X_train.shape[1],))

    h = layers.Dense(64, activation='relu')(x)
    h = layers.Dense(64, activation='relu')(h)
    h = layers.Dense(64, activation='relu')(h)
    h = layers.Dense(64, activation='relu')(h)

    y = layers.Dense(1, activation='sigmoid')(h)

    model = keras.Model(inputs=x, outputs=y)

    return model

def train_step(x, y, model, optimizer, loss_fn, train_acc_metric):
    with GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)

    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(y, logits)

    return loss_value

def test_step(x, y, model, loss_fn, val_acc_metric):
    val_logits = model(x, training=False)
    loss_value = loss_fn(y, val_logits)
    val_acc_metric.update_state(y, val_logits)

    return loss_value

Basierend auf den zuvor erstellten Funktionen wird nun die Trainingsfunktion erstellt. Dabei wird Weight & Biases verwendet, um den Trainingsprozess zu tracken.

In [7]:
def train(train_dataset,
          val_dataset, 
          model,
          optimizer,
          loss_fn,
          train_acc_metric,
          val_acc_metric,
          epochs=1000, 
          log_step=200, 
          val_log_step=50):
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))

        train_loss = []   
        val_loss = []

        # Iterate over the batches of the dataset
        for step, (x_batch_train, y_batch_train) in tqdm.tqdm(enumerate(train_dataset), total=len(train_dataset)):
            loss_value = train_step(x_batch_train, y_batch_train, 
                                    model, optimizer, 
                                    loss_fn, train_acc_metric)
            train_loss.append(float(loss_value))

        # Run a validation loop at the end of each epoch
        for step, (x_batch_val, y_batch_val) in enumerate(val_dataset):
            val_loss_value = test_step(x_batch_val, y_batch_val, 
                                       model, loss_fn, 
                                       val_acc_metric)
            val_loss.append(float(val_loss_value))
            
        # Display metrics at the end of each epoch
        train_acc = train_acc_metric.result()
        print("Training acc over epoch: %.4f" % (float(train_acc),))

        val_acc = val_acc_metric.result()
        print("Validation acc: %.4f" % (float(val_acc),))

        # Reset metrics at the end of each epoch
        train_acc_metric.reset_state()
        val_acc_metric.reset_state()

        # log metrics using wandb.log
        wandb.log({'epochs': epoch,
                   'loss': np.mean(train_loss),
                   'acc': float(train_acc), 
                   'val_loss': np.mean(val_loss),
                   'val_acc':float(val_acc)})

Nun werden die Parameter für die optimierung in der `sweep_config` definiert.

In [8]:
sweep_config = {
  'method': 'grid', 
  'metric': {
      'name': 'val_loss',
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },
  'parameters': {
      'batch_size': {
          'values': [32, 64, 128, 256, 512]
      }
  }
}

Anschließend wird die `sweep_train`-Funktion erstellt. Diese nutzt die zuvor definierte Trainingsfunktion und führt sie mit der `swep_config` zusammen, um den Optimierungsprozess in sogenannten Sweeps durchzuführen und auf Weight & Biases zu tracken.

In [9]:
def sweep_train(config_defaults=None):
    # Set default values
    config_defaults = {
        "batch_size": 32,
        "learning_rate": 0.001
    }
    # Initialize wandb with a sample project name
    wandb.init(reinit = True, config=config_defaults)  # this gets over-written in the Sweep

    # Specify the other hyperparameters to the configuration, if any
    wandb.config.epochs = 10
    wandb.config.log_step = 20
    wandb.config.val_log_step = 50
    wandb.config.architecture_name = "test"
    wandb.config.dataset_name = "Credit Card"

    # build input pipeline using tf.data
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
    train_dataset = (train_dataset.batch(wandb.config.batch_size)
                                  .prefetch(buffer_size=tf.data.AUTOTUNE)
                                  .shuffle(buffer_size=50176)
                                  )

    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val))
    val_dataset = (val_dataset.batch(wandb.config.batch_size)
                              .prefetch(buffer_size=tf.data.AUTOTUNE)
                              .shuffle(buffer_size=10752)
                              )

    # initialize model
    model = Model()

    # Instantiate an optimizer to train the model.
    # optimizer = keras.optimizers.SGD(learning_rate=wandb.config.learning_rate)
    optimizer = optimizers.Adam(learning_rate=wandb.config.learning_rate, 
                            beta_1=0.9, 
                            beta_2=0.999, 
                            amsgrad=True)
    # Instantiate a loss function.
    loss_fn = keras.losses.BinaryCrossentropy(from_logits=False)

    # Prepare the metrics.
    train_acc_metric = keras.metrics.BinaryAccuracy()
    val_acc_metric = keras.metrics.BinaryAccuracy()

    train(train_dataset,
          val_dataset, 
          model,
          optimizer,
          loss_fn,
          train_acc_metric,
          val_acc_metric,
          epochs=wandb.config.epochs, 
          log_step=wandb.config.log_step, 
          val_log_step=wandb.config.val_log_step)

Nun wird ein neuer Sweep erstellt und die `sweep_train`-Funktion aufgerufen, um den Optimierungsprozess zu starten.

In [10]:
sweep_id = wandb.sweep(sweep_config, project="credit_card")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: lj0f8np5
Sweep URL: https://wandb.ai/lokrau/Credit_Card/sweeps/lj0f8np5


In [11]:
wandb.agent(sweep_id, function=sweep_train, count=5)

wandb: Agent Starting Run: lwt2l64y with config:
wandb: 	batch_size: 32
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lokrau2002 (lokrau). Use `wandb login --relogin` to force relogin



Start of epoch 0


100%|██████████| 1568/1568 [00:31<00:00, 50.53it/s]
2024-06-09 11:21:15.327617: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Training acc over epoch: 0.5228
Validation acc: 0.5016

Start of epoch 1


100%|██████████| 1568/1568 [00:31<00:00, 49.27it/s]
2024-06-09 11:21:48.160029: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

100%|██████████| 98/98 [00:02<00:00, 41.94it/s]
2024-06-09 11:31:19.164641: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Training acc over epoch: 0.4574
Validation acc: 0.5172

Start of epoch 9


100%|██████████| 98/98 [00:02<00:00, 43.51it/s]
2024-06-09 11:31:21.516175: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Training acc over epoch: 0.4472
Validation acc: 0.4974


acc,▄▂▄▄▃▄█▄▂▁
epochs,▁▂▃▃▄▅▆▆▇█
loss,█▄▅▆▂▂▂▁▁▁
val_acc,▃▂▁▂▅▃▃█▅▂
val_loss,▃█▃▆▂▃▂▁▁▂
acc,0.44721
epochs,9
loss,33.58844
val_acc,0.4974
val_loss,52.84996


Nun werden die Ergebnisse dieses Sweeps geladen.

In [12]:
api = wandb.Api()
sweep_id = 'lokrau/credit_card/sweeps/' + sweep_id

sweep = api.sweep(sweep_id)
runs = sweep.runs

In [13]:
wandb.finish()

Diese ergebnisse werden in einer Liste gespeichert.

In [14]:
results = []
for run in runs:
    run_data = {
        'batch_size': run.config.get('batch_size'),
        'val_accuracy': run.summary.get('val_acc'),
        'val_loss': run.summary.get('val_loss'),
        'accuracy': run.summary.get('acc'),
        'loss': run.summary.get('loss'),
    }
    results.append(run_data)

In [15]:
results_df = pd.DataFrame(results)
results_df

,batch_size,val_accuracy,val_loss,accuracy,loss
0,512,0.497396,52.849962,0.447206,33.588444
1,256,0.510510,3.600855,0.461535,13.944678
2,128,0.523717,1.654581,0.532605,1.178922
3,64,0.551246,1.345409,0.533721,3.202749
4,32,0.505301,0.853156,0.522999,0.753955


Nun kann daraus der beste Run und damit die beste Batch Size ermittelt werden.

In [16]:
best_run = results_df.loc[results_df['val_loss'].idxmin()]
best_batch_size = best_run['batch_size']
best_val_loss = best_run['val_loss']

In [17]:
print(f'Best batch size: {best_batch_size} with validation loss: {best_val_loss}')

Best batch size: 32.0 with validation loss: 0.8531560952819529
